In [0]:
from __future__ import print_function
from tensorflow.python.keras.callbacks import LambdaCallback
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io
from tensorflow.python.keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
import gensim
from gensim.models import KeyedVectors
import string

In [0]:
# !wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [0]:
# EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
# word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [0]:
max_sentence_len = 40
with open('pap.txt') as f_in:
    texts = list(line for line in (l.strip() for l in f_in) if line)

In [5]:
print(texts[0])

PRIDE AND PREJUDICE


In [0]:
with open('text_gen_test.txt') as file_:
    testingTexts = file_.readlines()

In [7]:
print(testingTexts)

['Elizabeth wanted to run away in the distance because\n', 'Mr. Darcy was a very\n', 'Mrs. Bennet was so excited that she could\n', 'Mr. Wickham, a tall and dashing young man, made\n', 'Lady Catherine, having heard rumours about Elizabeth and Darcy, visits']


In [0]:
# pre_sentences =[[word for word in text.lower().split()[:max_sentence_len]if word in word2vec.vocab] for text in texts]
pre_sentences =[[word for word in text.lower().split()[:max_sentence_len]] for text in texts]

In [0]:
word2vec = gensim.models.Word2Vec(pre_sentences, size=100, min_count=1, 
                                    window=5, iter=200)

In [10]:
pretrained_weights = word2vec.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)

Result embedding shape: (8280, 100)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [0]:
pre_sentences_2 =[[word for word in text.lower().split()[:max_sentence_len]if word in word2vec.wv.vocab] for text in texts]

In [0]:
sentences = []
for sentence in pre_sentences_2:
#   print(sentence)
  if len(sentence)!=0:
    sentences.append(sentence)
  

In [13]:
print('Num sentences:', len(sentences))
print(sentences[14])

Num sentences: 2120
['“bingley.”']


In [0]:
def word2idx(word):
   return word2vec.wv.vocab[word].index
def idx2word(idx):
  return word2vec.wv.index2word[idx]

In [15]:
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)

train_x shape: (2120, 40)
train_y shape: (2120,)


In [0]:
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
#   print(sentence1[-1])
  train_y[i] = word2idx(sentence[-1])

In [17]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
def on_epoch_end(epoch, _):
  print('\nGenerating text after epoch: %d' % epoch)
  
  for text in testingTexts[:2]:
    sample = generate_next(text,0.5)
    print('%s... -> %s' % (text, sample))


In [0]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

def generate_next(text,diversity):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(10):
    prediction = model.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], diversity)
    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)

In [20]:
model.fit(train_x, train_y,
          batch_size=128,
          epochs=20,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Epoch 1/20
2048/2120 [===========================>..] - ETA: 0s - loss: 8.9079
Generating text after epoch: 0
Elizabeth wanted to run away in the distance because
... -> elizabeth wanted to run away in the distance because removal.” pouring abominably depended; justice, persons as, least.” elizabeth,” neighbour
Mr. Darcy was a very
... -> mr. darcy was a very 45 dissatisfied, mortifying efficacy delicacy, gradually mean,” licence. commission circulation.
2120/2120 [==============================] - 6s 3ms/sample - loss: 8.8979
Epoch 2/20
2048/2120 [===========================>..] - ETA: 0s - loss: 7.5739
Generating text after epoch: 1
Elizabeth wanted to run away in the distance because
... -> elizabeth wanted to run away in the distance because amongst spared could, lost. will happier mature wonders, revived. interesting
Mr. Darcy was a very
... -> mr. darcy was a very terms, esteemed love. breast officers' refuse contentment, introduced fitzwilliam's tuesday,
2120/2120 [=============